In [1]:
# This would be impossible in any other language
from typing import List, Iterable, Any
from itertools import product, combinations_with_replacement

In [2]:
def n_iterator(iterables: List[Iterable[Any]], length: int, use_product: bool = True) -> Iterable[Any]:
    """
    Create the appropriate combinatorial iterator over a collection of iterables.
    The iterator will return a total of `length` items.
    The length of `iterables` can be `length`, in which case each iterable is used for one value.
    The length of `iterables` can be 1, in which case the same iterable is use for all values.
    If the length of `iterables` is 1, can specify whether to use the catesian product of
    iterables, or to take combinations with replacement.
    
    Args:
        iterables:   List of iterables, length either 1 or `length`.
        length:      Desired number of outputs from return iterator.
        use_product: If `len(iterables) == 1`, use cartesian product.
    
    Returns:
        Iterator with `length` return values.
    """
    
    # Ensure `len(iterables)` is either 1 or `length`
    if not (len(iterables) == 1 or len(iterables) == length):
        raise ValueError(f"Number of iterables must be either one (1) or `length` ({length}).")
    
    # Can only use combinations_w_r if `len(iterables) == 1`
    if not use_product and len(iterables) != 1:
        raise ValueError("Cannot set `use_product` to false unless only one iterable is passed.")
    
    # Select the appropriate combinatorial iterator
    if len(iterables) > 1:
        return product(*iterables)
    if use_product:
        return product(iterables[0], repeat=length)
    return combinations_with_replacement(iterables[0], r=length)

In [3]:
def one_plus_n_generator(one: Iterable[Any], iterables: List[Iterable[Any]], length: int, use_product: bool = True) -> Iterable[Any]:
    """
    Create the appropriate combinatorial iterator over a collection of 1+n iterables.
    View n_iterator.__doc__ for a description of the parameters `iterables`, `length`, `use_product`.
    The cartesian product is taken between the one and n iterables.
    Enumerate is used on all iterables to ensure 
    
    Args:
        one:         Single iterable.
        iterables:   List of iterables, length either 1 or `length`.
        length:      Desired number of outputs from return iterator.
        use_product: If `len(iterables) == 1`, use cartesian product.
    
    Returns:
        Generator yielding one_id, one_val, n_ids, n_vals
    """
    
    # Create enumeraters for all iteraters
    enum_one  = enumerate(one)
    enum_iter = [enumerate(iterable) for iterable in iterables]
    
    # Retrieve the n-iterator
    enum_n_iter = n_iterator(enum_iter, length, use_product)
    
    # Run cartesian product between the 1 and n iterables
    for ((one_id, one_val), n_ids_vals) in  product(enum_one, enum_n_iter):
        
        # Yield indices and values for 1 + n iterables
        n_ids  = [n[0] for n in n_ids_vals]
        n_vals = [n[1] for n in n_ids_vals]
        yield one_id, one_val, n_ids, n_vals

In [4]:
# Define some sample lists
r = ['r0', 'r1']
a = ['a0', 'a1']
b = ['b0', 'b1']

In [5]:
# Case A)
for one_id, one_val, n_ids, n_vals in one_plus_n_generator(r, [a, b], length=2):
    print(one_id, n_ids, one_val, n_vals)

0 [0, 0] r0 ['a0', 'b0']
0 [0, 1] r0 ['a0', 'b1']
0 [1, 0] r0 ['a1', 'b0']
0 [1, 1] r0 ['a1', 'b1']
1 [0, 0] r1 ['a0', 'b0']
1 [0, 1] r1 ['a0', 'b1']
1 [1, 0] r1 ['a1', 'b0']
1 [1, 1] r1 ['a1', 'b1']


In [6]:
# Case B)
for one_id, one_val, n_ids, n_vals in one_plus_n_generator(r, [a], length=2, use_product=True):
    print(one_id, n_ids, one_val, n_vals)

0 [0, 0] r0 ['a0', 'a0']
0 [0, 1] r0 ['a0', 'a1']
0 [1, 0] r0 ['a1', 'a0']
0 [1, 1] r0 ['a1', 'a1']
1 [0, 0] r1 ['a0', 'a0']
1 [0, 1] r1 ['a0', 'a1']
1 [1, 0] r1 ['a1', 'a0']
1 [1, 1] r1 ['a1', 'a1']


In [7]:
# Case C)
for one_id, one_val, n_ids, n_vals in one_plus_n_generator(r, [a], length=2, use_product=False):
    print(one_id, n_ids, one_val, n_vals)

0 [0, 0] r0 ['a0', 'a0']
0 [0, 1] r0 ['a0', 'a1']
0 [1, 1] r0 ['a1', 'a1']
1 [0, 0] r1 ['a0', 'a0']
1 [0, 1] r1 ['a0', 'a1']
1 [1, 1] r1 ['a1', 'a1']
